## Installing Required Libraries

In [1]:
!pip install openai chromadb sentence_transformers -q

In [3]:
!pip install langchain unstructured -q 

## Load Documents

In [4]:
from langchain.document_loaders import DirectoryLoader
directory = 'countries'

def load_doc(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_doc(directory)
len(documents)

5

## Splitting to Chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_doc(documents,chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_doc(documents)
print(len(docs))

23


In [8]:
docs[0].page_content

"Let's explore the culture, festivals, and history of England in detail.\n\n\n\n\n\nEngland's Culture:**\n\n**1. Cultural Heritage:** England is renowned for its rich cultural heritage, which has significantly influenced the world. Its contributions include literature, theater, music, and art. Notable figures include William Shakespeare, Charles Dickens, and The Beatles.\n\n**2. Language:** English, the official language, has become a global lingua franca. England's language and literature have played a pivotal role in shaping the modern world.\n\n**3. Arts and Entertainment:** England is home to world-class theaters like London's West End, where numerous plays and musicals are performed. The country has a strong tradition of classical music and hosts events like the BBC Proms.\n\n**4. Cuisine:** English cuisine includes classics like fish and chips, roast dinners, and afternoon tea. The country also has a diverse culinary scene influenced by immigration, with a variety of internationa

In [9]:
docs[22].page_content

"Pakistan's culture, festivals, and history are deeply rooted in its Islamic heritage and diverse cultural influences. The country's rich traditions and historical legacy contribute to its unique identity."

## Embedding 

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


## Vector Database

In [11]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

## Similarity Search

In [13]:
query = "What is the culture of India"
matching_docs = db.similarity_search(query,k=1)
matching_docs

[Document(page_content='India is a vast and diverse country with a rich cultural heritage, a tapestry of traditions, and a history that spans thousands of years. Let\'s delve into India\'s culture, festivals, and history in detail.\n\n\n\n\n\nIndia\'s Culture:**\n\n**1. Cultural Diversity:** India is often referred to as the "Land of Diversity" due to its incredible variety of cultures, languages, religions, and traditions. It is home to numerous ethnic groups, each with its unique customs and practices.\n\n**2. Religion:** India is the birthplace of major religions such as Hinduism, Buddhism, Jainism, and Sikhism. It\'s also home to significant populations of Muslims, Christians, and other faiths. The religious diversity has a profound influence on India\'s cultural landscape.', metadata={'source': 'countries/india.txt'})]

In [14]:
## Similarity search with function

def similarity_docs(query,k=2,score=False):
    if score:
        similar_docs = db.similarity_search_with_score(query,k=k)
    else:
        similar_docs= db.similarity_search(query,k=k)
    return similar_docs

In [17]:
query = "what are the festivals of USA"
similar_docs = similarity_docs(query,score=True)
similar_docs

[(Document(page_content="The United States' culture, festivals, and history reflect its diverse and dynamic nature. It has shaped and been shaped by waves of immigration, social movements, and technological advancements, making it a complex and influential nation on the global stage.", metadata={'source': 'countries/USA.txt'}),
  0.6867135763168335),
 (Document(page_content="**4. Cuisine:** American cuisine is a reflection of its diverse population. It includes dishes influenced by Native American, European, African, Asian, and Latin American traditions. Fast food, barbecue, and regional specialties like Tex-Mex and New England clam chowder are notable.\n\n**5. Clothing:** American clothing styles vary widely, from casual wear like jeans and T-shirts to formal attire for business and special occasions. The U.S. is also known for its fashion industry, with cities like New York being major fashion hubs.\n\n\n\n\n\nUSA's Festivals:**\n\n**1. Independence Day:** The Fourth of July is a sig

## Perstist VectorDB instance 

In [18]:
perstist_directory = "db"

vectordb = Chroma.from_documents(
    documents=docs,embedding=embeddings,persist_directory=perstist_directory
)

In [19]:
vectordb.persist()

## Load persisted Directory

In [20]:
chroma_db = Chroma(persist_directory='db/',embedding_function=embeddings)

In [21]:
query = "what is the culture of Canada"
result = chroma_db.similarity_search(query)

In [22]:
result

[Document(page_content="Let's explore the culture, festivals, and history of Canada in detail.\n\n\n\n\n\nCanada's Culture:**\n\n**1. Cultural Diversity:** Canada is known for its cultural mosaic, where people from diverse backgrounds and ethnicities coexist harmoniously. This multiculturalism is a defining feature of Canadian society.\n\n**2. Bilingualism:** Canada is officially bilingual, with English and French as the two official languages. This bilingualism reflects the country's history and the presence of a significant French-speaking population in the province of Quebec.\n\n**3. Arts and Culture:** Canada has a rich artistic heritage, with contributions to literature, music, film, and visual arts. Canadian authors like Margaret Atwood and Yann Martel have achieved international acclaim. The country is also known for its indigenous art and traditions.", metadata={'source': 'countries/canada.txt'}),
 Document(page_content="**4. Indigenous Culture:** Canada's indigenous peoples, i

## LLM

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('openai_key')

In [ ]:
print(OPENAI_API_KEY)

In [27]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

## Getting Answer

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type='stuff',verbose=True)

query = "how is the clothing of pakistan"
similar_docs = similarity_docs(query,k=2)
chain.run(input_documents=similar_docs,question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Let's explore Pakistan's culture, festivals, and history in detail.





Pakistan's Culture:**

**1. Cultural Diversity:** Pakistan is a diverse country with a rich tapestry of cultures, languages, and traditions. It is home to various ethnic groups, each contributing to the nation's unique cultural mosaic.

**2. Religion:** Islam is the predominant religion in Pakistan, and it plays a central role in shaping the country's culture and way of life. Mosques, Islamic art, and calligraphy are integral parts of the cultural landscape.

**3. Art and Architecture:** Pakistan has a rich artistic heritage, with impressive Islamic architecture, including Badshahi Mosque in Lahore and Shah Jahan Mosque in Thatta

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_ret

RateLimitError: You exceeded your current quota, please check your plan and billing details.